In [ ]:
import psutil
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense , Flatten 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop
import time
import random

# --- Configuration & Seeding ---
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

JSON_FILE_PATH = "imdb_dataset_prepared.json"
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_TRAINABLE = True
BATCH_SIZE = 128
EPOCHS = 12

In [ ]:
# Function to get current process memory
def get_process_memory():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss  

# Custom callback to monitor memory
class MemoryCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        if epoch + 1 == EPOCHS:
            print(f"\tEpoch {epoch+1}: End memory usage: {get_process_memory() / (1024**2):.2f} MB")

In [8]:
with open(JSON_FILE_PATH, 'r') as f:
    data = json.load(f)

X_train = np.array(data['X_train'])
y_train = np.array(data['y_train'])
X_test = np.array(data['X_test'])
y_test = np.array(data['y_test'])

X_train -= 1
X_test -= 1

embeddings = np.array(data['embeddings'])
vocab = data['vocab']
embedding_dim = embeddings.shape[1]

In [9]:
print(f"Initial memory usage: {get_process_memory() / (1024**2):.2f} MB")

model = Sequential()
model.add(Embedding(len(vocab), embedding_dim, weights=[embeddings]))
model.add(SimpleRNN(16, 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

print(f"Memory after model compilation: {get_process_memory() / (1024**2):.2f} MB")

Initial memory usage: 758.43 MB


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │       642,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 642,450 (2.45 MB)

 Trainable params: 642,450 (2.45 MB)

 Non-trainable params: 0 (0.00 B)

Memory after model compilation: 760.99 MB


In [10]:
print(f"\nTraining for {EPOCHS} epochs with batch size {BATCH_SIZE}...")
start_time_train = time.time()
history = model.fit(X_train, y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(X_test, y_test),
                    shuffle=True,
                    verbose=0,
                    callbacks=[MemoryCallback()])
training_time = time.time() - start_time_train
print(f"Training completed in {training_time:.2f}s")
print(f"Final memory usage: {get_process_memory() / (1024**2):.2f} MB")

print("\n--- Training History ---")
for epoch in range(EPOCHS):
    train_loss = history.history['loss'][epoch]
    train_acc = history.history['accuracy'][epoch]
    val_loss = history.history['val_loss'][epoch]
    val_acc = history.history['val_accuracy'][epoch]
    time_per_epoch_approx = training_time / EPOCHS
    print(f"Epoch: {epoch+1}/{EPOCHS} ({time_per_epoch_approx:.2f}s approx.) "
          f"\tTrain: (l: {train_loss:.2f}, a: {train_acc:.2f}) "
          f"\tTest: (l: {val_loss:.2f}, a: {val_acc:.2f})")

print("\n--- Final Evaluation on Test Set ---")
final_loss, final_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {final_loss:.4f}")
print(f"Test Accuracy: {final_accuracy:.4f}")


Training for 12 epochs with batch size 128...
	Epoch 12: End memory usage: 856.93 MB
Training completed in 202.77s
Final memory usage: 806.91 MB

--- Training History ---
Epoch: 1/12 (16.90s approx.) 	Train: (l: 0.70, a: 0.51) 	Test: (l: 0.69, a: 0.50)
Epoch: 2/12 (16.90s approx.) 	Train: (l: 0.69, a: 0.52) 	Test: (l: 0.69, a: 0.51)
Epoch: 3/12 (16.90s approx.) 	Train: (l: 0.69, a: 0.53) 	Test: (l: 0.69, a: 0.52)
Epoch: 4/12 (16.90s approx.) 	Train: (l: 0.68, a: 0.55) 	Test: (l: 0.67, a: 0.57)
Epoch: 5/12 (16.90s approx.) 	Train: (l: 0.62, a: 0.68) 	Test: (l: 0.60, a: 0.71)
Epoch: 6/12 (16.90s approx.) 	Train: (l: 0.56, a: 0.73) 	Test: (l: 0.52, a: 0.75)
Epoch: 7/12 (16.90s approx.) 	Train: (l: 0.53, a: 0.76) 	Test: (l: 0.54, a: 0.76)
Epoch: 8/12 (16.90s approx.) 	Train: (l: 0.50, a: 0.78) 	Test: (l: 0.46, a: 0.78)
Epoch: 9/12 (16.90s approx.) 	Train: (l: 0.47, a: 0.80) 	Test: (l: 0.42, a: 0.81)
Epoch: 10/12 (16.90s approx.) 	Train: (l: 0.43, a: 0.81) 	Test: (l: 0.43, a: 0.78)
Epoch: 